In [585]:
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np

In [586]:
driver=webdriver.Chrome("C:/Users/Playdata/Desktop/chromedriver_win32/chromedriver.exe")
url="https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc"

C:\Users\Playdata\AppData\Local\Temp\ipykernel_7008\1645652240.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("C:/Users/Playdata/Desktop/chromedriver_win32/chromedriver.exe")


In [10]:
driver.get(url)

In [15]:
html=driver.page_source
len(html)

616217

In [16]:
soup=BeautifulSoup(html,'html.parser')
print(soup.prettify)

<bound method Tag.prettify of <html class="scriptsOn" style="--ipt-focus-outline-on-base: none; --ipt-focus-outline-on-baseAlt: none;" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#"><head><script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/31bJewCvY-L.js"></script>
<script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b

In [75]:
# movie_div=soup.find_all('div',class_='lister-item mode-advanced')
# for container in movie_div:
#     name=container.h3.a.text
#     print(name)

쓰리 빌보드
스포트라이트
Room
Lock, Stock and Two Smoking Barrels
Monsters, Inc.
사운드 오브 뮤직
Salinui chueok
그랜 토리노
드래곤 길들이기
Platoon
The Deer Hunter
비포 선라이즈
Warrior
Rush
12 Years a Slave
Raging Bull
Waildeu Teiljeu: Cham-eul Su Eobsneun Sungan
밀리언 달러 베이비
The Iron Giant
Barry Lyndon
Hachi: A Dog's Tale
La haine
Tonari no Totoro
Network
Ben-Hur
Before Sunset
On the Waterfront
Hotel Rwanda
The Seventh Seal
Amores perros
In the Name of the Father
The Bridge on the River Kwai
Persona
Rebecca
어느 날 밤에 생긴 일
The Grapes of Wrath
The Third Man
Mary and Max
The Best Years of Our Lives
The Gold Rush
The 400 Blows
Smultronstället
Mr. Smith Goes to Washington
La battaglia di Algeri
The General
The Incredibles
Dances with Wolves
알라딘
사랑의 블랙홀
Life of Brian


In [236]:
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')

In [290]:
one_movie_info = soup.select_one('div.lister-item.mode-advanced')
one_movie_info.h3.a['href'].split('/')[2]

'tt5027774'

In [296]:
one_movie_info.find('span',class_='genre').text.strip()

'Comedy'

In [752]:
import time
import numpy as np
names=[]
years=[]
runtimes=[]
imdb_ratings=[]
meta_scores=[]
genres=[]
movie_id=[]
pages=np.arange(1,251,50)
for page in pages:
    url=f'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start={page}&ref_=adv_nxt'
    time.sleep(3)
    driver.get(url)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    soup.prettify
    movies_info = soup.select('div.lister-item.mode-advanced')
    #이름
    for one_movie_info in movies_info:
        #제목
        names.append(one_movie_info.h3.a.get_text().strip())
        #연도
        years.append(one_movie_info.h3.find('span',class_='lister-item-year').text)
        #시간
        runtimes.append(one_movie_info.p.find('span',class_='runtime').text)
        #평점
        imdb_ratings.append(float(one_movie_info.strong.text))
        #메타스코어 np.nan
        if one_movie_info.find('span',class_='metascore') == None:
            meta_scores.append(np.nan)
        else:
            meta_scores.append(int(one_movie_info.find('span',class_='metascore').get_text().replace(' ','')))
        genres.append(one_movie_info.find('span',class_='genre').text.strip())
        movie_id.append(one_movie_info.h3.a['href'].split('/')[2])
            

        
        # 하나의 영화씩 선택 -> one_movie_info
        # one_movie_info.select_one('h3>a')
        # 연도
        # 시간
        # 평점
        # 메타스코어 np.nan
        # 5개의 원소가 들어가게

    # #movie_year
    # movie_year=soup.select('h3>span.lister-item-year')
    # for year in movie_year:
    #     years.append(year.get_text().strip())
    # #time
    # run_time=soup.select('p>span.runtime')
    # for runtime in run_time:
    #     runtimes.append(runtime.get_text().strip())
    # #strong(평점)
    # imdb_rating=soup.select('strong')[2:]
    # for rating in imdb_rating:
    #     imdb_ratings.append(float(rating.get_text().strip()))
    # #메타스코어(점수)
    
    # meta_score=soup.select('span.metascore')
    # for metascore in meta_score:
    #     meta=metascore.get_text().strip()
    #     if len(meta)>1:
    #         meta_scores.append(int(meta))
    #     else:
    #         meta_scores.append(np.nan)
    # #장르
    # genress=soup.select('span.genre')
    # for genre in genress:
    #     genres.append(genre.get_text().strip())



In [706]:
genres
len(movie_id)

250

In [116]:
meta_score=soup.select('span.metascore')
int(meta_score[0].get_text().strip())

88

In [754]:
df=pd.DataFrame([movie_id,names,years,runtimes,imdb_ratings,meta_scores,genres]).T
df.columns=['movie_id','name','year','runtime(min)','imdb_rating','meta_score','genre']
df['year']=df['year'].apply(lambda x : x.replace("(","").replace(")",""))
df['runtime(min)']=df['runtime(min)'].apply(lambda x: x[:3])
df['meta_score'].value_counts()

86     12
90     12
84      9
77      9
88      9
98      9
92      9
85      8
96      8
75      8
100     7
89      7
82      7
94      7
97      6
79      6
87      6
78      6
81      6
72      5
71      5
95      5
65      4
69      4
93      4
83      4
74      4
66      4
73      4
91      4
70      4
63      3
76      3
80      3
68      3
62      3
64      3
99      2
57      2
67      2
59      2
58      1
60      1
55      1
61      1
Name: meta_score, dtype: int64

In [346]:
df['meta_score'].isnull().sum()

17

In [347]:

# movie_year=soup.select('h3>span.lister-item-year')
# for year in movie_year:
#     years.append(year.get_text().strip())
# years

df.to_csv('movie_info.csv',index=False)

In [126]:
movie_name=soup.select('strong')[2:]
for rating in movie_name:
    rating.get_text().strip()


[<strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.1</strong>,
 <strong>8.

In [120]:
meta_score=soup.select('span.metascore')
for metascore in meta_score:
    meta=metascore.get_text().strip()

'77'

In [128]:
genres=soup.select('span.genre')
for genre in genres:
    print(genre.get_text().strip())


Comedy, Crime, Drama
Biography, Crime, Drama
Drama, Thriller
Action, Comedy, Crime
Animation, Adventure, Comedy
Biography, Drama, Family
Crime, Drama, Mystery
Drama
Animation, Action, Adventure
Drama, War
Drama, War
Drama, Romance
Action, Drama, Sport
Action, Biography, Drama
Biography, Drama, History
Biography, Drama, Sport
Comedy, Drama, Thriller
Drama, Sport
Animation, Action, Adventure
Adventure, Drama, War
Biography, Drama, Family
Crime, Drama
Animation, Comedy, Family
Drama
Adventure, Drama
Drama, Romance
Crime, Drama, Thriller
Biography, Drama, History
Drama, Fantasy
Drama, Thriller
Biography, Crime, Drama
Adventure, Drama, War
Drama, Thriller
Drama, Film-Noir, Mystery
Comedy, Romance
Drama
Film-Noir, Mystery, Thriller
Animation, Comedy, Drama
Drama, Romance, War
Adventure, Comedy, Drama
Crime, Drama
Drama, Romance
Comedy, Drama
Drama, War
Action, Adventure, Comedy
Animation, Action, Adventure
Adventure, Drama, Western
Animation, Adventure, Comedy
Comedy, Drama, Fantasy
Comedy


In [247]:
filtering=soup.select('h3>a')
for filter in filtering:
        print(filter['href'].split('/')[2])

tt5027774
tt1895587
tt3170832
tt0120735
tt0198781
tt0059742
tt0353969
tt1205489
tt0892769
tt0091763
tt0077416
tt0112471
tt1291584
tt1979320
tt2024544
tt0081398
tt3011894
tt0405159
tt0129167
tt0072684
tt1028532
tt0113247
tt0096283
tt0074958
tt0052618
tt0381681
tt0047296
tt0395169
tt0050976
tt0245712
tt0107207
tt0050212
tt0060827
tt0032976
tt0025316
tt0032551
tt0041959
tt0978762
tt0036868
tt0015864
tt0053198
tt0050986
tt0031679
tt0058946
tt0017925
tt0317705
tt0099348
tt0103639
tt0107048
tt0079470


In [714]:
review_info=soup.select('#main div.review-container > div.lister-item-content')
listreview=[]
# > div.ipl-ratings-bar'
# rating_info=soup.select('div.lister-item-content > div.ipl-ratings-bar')
# review_info.select_one('div.ipl-ratings-bar').get_text().strip()
for reviewinfo in review_info:
    if reviewinfo.select('div.ipl-ratings-bar')==[]:
        print(np.nan)
    else:
        n=int(reviewinfo.select_one('div.ipl-ratings-bar').get_text().strip().split('/')[0])
        print(n)


# len(review_info)

# n=soup.select('.rating-other-user-rating>span')
# for i in range(0,len(n),2):
#     rating.append(int(n[i].get_text()))
# rating
# rating
# review_info.div
# review_info.find('span',class_='display-name-link').text
# review_info.select_one('span.display-name-link').get_text()

10
10
7
10
10
10
10
8
10
8
10
10
10
8
9
8
10
8
8
9
10
10
10
10
10


In [711]:
len(review_info[:10])

10

In [716]:
id2=[]
user_id=[]
rating=[]
for id in movie_id:
    url="https://www.imdb.com/title/"+id+"/reviews/?ref_=tt_ql_2"
    driver.get(url)
    time.sleep(3)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    review_info=soup.select('#main div.review-container')
    rating_info=soup.select('div.lister-item-content > div.ipl-ratings-bar')
    for j in range(len(review_info[:10])):
        id2.append(id)
        # user_ids=review_info.find('span',class_='display-name-link').text
        user_ids=review_info[j].select_one('span.display-name-link').get_text()
        user_id.append(user_ids)
        # rating
        review_info=soup.select('#main div.review-container > div.lister-item-content')[:10]
    for reviewinfo in review_info:
        if reviewinfo.select('div.ipl-ratings-bar')==[]:
            rating.append(np.nan)
        else:
            rating.append(int(reviewinfo.select_one('div.ipl-ratings-bar').get_text().strip().split('/')[0]))
    
        
    

In [794]:
df2=pd.DataFrame([id2,user_id,rating]).T
df2.columns=['movie_id','user_id','rating']
df2['user_id'].value_counts()
# # df2.loc[df2['user_id']=='TheLittleSongbird'][['rating']]
# df2.to_csv('useer_info.csv',index=False)
df2.head(10)
df

,movie_id,name,year,runtime(min),imdb_rating,meta_score,genre
0,tt0111161,The Shawshank Redemption,1994,142,9.3,82,Drama
1,tt0068646,The Godfather,1972,175,9.2,100,"Crime, Drama"
2,tt0468569,The Dark Knight,2008,152,9.0,84,"Action, Crime, Drama"
3,tt0108052,Schindler's List,1993,195,9.0,95,"Biography, Drama, History"
4,tt0050083,12 Angry Men,1957,96,9.0,97,"Crime, Drama"
...,...,...,...,...,...,...,...
245,tt0317705,The Incredibles,2004,115,8.0,90,"Animation, Action, Adventure"
246,tt0099348,Dances with Wolves,1990,181,8.0,72,"Adventure, Drama, Western"
247,tt0103639,Aladdin,1992,90,8.0,86,"Animation, Adventure, Comedy"
248,tt0107048,Groundhog Day,1993,101,8.0,72,"Comedy, Drama, Fantasy"


In [796]:
combined=pd.merge(df2,df)
combined
pvt=combined.pivot_table(index='user_id',columns='name',values='rating').fillna(0)
pvt
item_corr=pvt.corr()
item_corr
combined.to_csv('movie_totalinfo.csv',index=False)

In [761]:
interested="Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb"
item_corr.sort_values(by=interested,ascending=False)[interested]

name
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb    1.000000
Life of Brian                                                           0.403802
Per qualche dollaro in più                                              0.354411
The Godfather                                                           0.340067
Ikiru                                                                   0.303684
                                                                          ...   
Interstellar                                                           -0.009176
Dances with Wolves                                                     -0.009176
Top Gun: Maverick                                                      -0.009179
Forrest Gump                                                           -0.009179
Rocky                                                                  -0.009179
Name: Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb, Length: 249, dtype: float64

In [762]:
user_corr=pvt.T.corr()
user_corr

user_id,0w0,851222,ACitizenCalledKane,ACollegeStudent,AKS-6,ASuiGeneris,A_Different_Drummer,Aaron1375,AaronCapenBanner,Abdulxoxo,...,yocarlosvarelapr,yummyballerina,yuschaa,zeki-4,zeletto,zeph-3,zetes,zkonedog,zolaaar,ztruk2001
user_id,,,,,,,,,,,,,,,,,,,,,
0w0,1.000000,-0.007004,-0.008080,-0.004032,-0.004032,-0.004032,0.404112,-0.004032,-0.006959,-0.004032,...,-0.004032,-0.004032,-0.004032,-0.004032,-0.004032,-0.004032,-0.009908,-0.009909,-0.004032,-0.004032
851222,-0.007004,1.000000,-0.014034,-0.007004,-0.007004,-0.007004,-0.017332,-0.007004,-0.012088,-0.007004,...,-0.007004,-0.007004,-0.007004,-0.007004,-0.007004,-0.007004,-0.017210,-0.017213,-0.007004,-0.007004
ACitizenCalledKane,-0.008080,-0.014034,1.000000,-0.008080,-0.008080,-0.008080,-0.019994,-0.008080,-0.013945,-0.008080,...,-0.008080,-0.008080,-0.008080,-0.008080,-0.008080,-0.008080,-0.019853,-0.019856,-0.008080,-0.008080
ACollegeStudent,-0.004032,-0.007004,-0.008080,1.000000,-0.004032,-0.004032,-0.009978,-0.004032,-0.006959,-0.004032,...,-0.004032,-0.004032,-0.004032,-0.004032,-0.004032,-0.004032,-0.009908,-0.009909,-0.004032,-0.004032
AKS-6,-0.004032,-0.007004,-0.008080,-0.004032,1.000000,-0.004032,-0.009978,-0.004032,-0.006959,-0.004032,...,-0.004032,-0.004032,-0.004032,-0.004032,-0.004032,-0.004032,-0.009908,-0.009909,-0.004032,-0.004032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zeph-3,-0.004032,-0.007004,-0.008080,-0.004032,-0.004032,-0.004032,-0.009978,-0.004032,-0.006959,-0.004032,...,-0.004032,-0.004032,-0.004032,-0.004032,-0.004032,1.000000,-0.009908,-0.009909,-0.004032,-0.004032
zetes,-0.009908,-0.017210,-0.019853,-0.009908,-0.009908,-0.009908,-0.024518,-0.009908,-0.017100,-0.009908,...,-0.009908,-0.009908,-0.009908,-0.009908,-0.009908,-0.009908,1.000000,-0.024349,-0.009908,-0.009908
zkonedog,-0.009909,-0.017213,-0.019856,-0.009909,-0.009909,-0.009909,-0.024522,-0.009909,-0.017103,-0.009909,...,-0.009909,-0.009909,-0.009909,-0.009909,-0.009909,-0.009909,-0.024349,1.000000,-0.009909,-0.009909


In [834]:
target='TheLittleSongbird'
tmp=user_corr.sort_values(by=target,ascending=False)[target]


user_id
TheLittleSongbird     1.000000
Smells_Like_Cheese    0.195598
FiendishDramaturgy    0.135078
RedRoadster           0.134890
dromasca              0.134843
                        ...   
perica-43151         -0.108804
classicsoncall       -0.128752
Xstal                -0.137248
Fella_shibby         -0.144085
Nazi_Fighter_David   -0.155197
Name: TheLittleSongbird, Length: 1042, dtype: float64

0       False
1       False
2       False
3       False
4       False
        ...  
2495    False
2496    False
2497    False
2498    False
2499    False
Name: user_id, Length: 2500, dtype: bool

In [842]:
a=tmp[tmp < 0.9999]
b=pd.DataFrame(a).reset_index()
list(b['user_id'].head(10))



['dtb',
 'dr_foreman',
 'TheLittleSongbird',
 'inspectors71',
 'Atreyu_II',
 'dcldan',
 'yocarlosvarelapr',
 'dxia',
 'aimless-46',
 'comix-man']

In [801]:
target='TheLittleSongbird'
user_corr.sort_values(by=target,ascending=False)[target].head()
u1=set(combined.loc[combined['user_id']=='TheLittleSongbird']['name'])
u2=set(combined.loc[combined['user_id']=='Smells_Like_Cheese']['name'])
u3=set(combined.loc[combined['user_id']=='boatsfra']['name'])
diff=u2.difference(u1)
u2_all=combined.loc[combined['user_id']=='Smells_Like_Cheese']
filtered=u2_all.loc[combined['name'].isin(diff)]
filtered.sort_values(by='rating',ascending=False).head()['name']




21                     The Dark Knight
90                        Forrest Gump
120    Il buono, il brutto, il cattivo
234           The Silence of the Lambs
420                        The Pianist
Name: name, dtype: object

In [591]:
soup.select_one('div.lister-item-content').get_text()
n=list(soup.select('div.lister-item-content').strings)[8]
n

'10'

In [574]:
# len(id2)
# len(user_id)
# len(rating)
soup.select('.rating-other-user-rating>span')

[<span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="point-scale">/10</span>,
 <span>9</span>,
 <span class="point-scale">/10</span>,
 <span>10</span>,
 <span class="

In [425]:
#main > section > div.lister > div.lister-list > div:nth-child(34) > div.review-container > div.lister-item-content > div.ipl-ratings-bar > span > span:nth-child(2)

25